# Holiday-planning: a teaser on OR modelling and implementation
This notebook accompanies a workshop on OR problem-solving, covering the following topics:
- understanding and refining the problem (in workshop)
- modelling: translating the problem into a mathematical model (in workshop)
- implementation: setting up the model using Python and PuLP, and using a solver to find the optimal solution (this notebook)
- result interpretation: understanding the output, visualizing the result and deriving insights (this notebook)

In [ ]:
# Import required dependencies (both 3rd party packages and custom modules)
import pulp
import os
import pandas as pd
from enum import Enum
from datetime import date, timedelta
import matplotlib.pyplot as plt
from typing import Callable
import sys 

sys.path.insert(0, '..')
from src.plotting.calendar import holiday_calendar_plot
from src.mdl.period import HolidayPeriod

In [ ]:
# Data import can be controlled by using the enum class
class Location(Enum):
    BERLIN = 'Berlin'
    MUNICH = 'Munich'
    
location = Location.MUNICH
year = 2025

filename_public_holidays = f'{year}_public_holidays_{location.value}.csv'
data_directory = os.path.join('..', 'datasets')

filepath_public_holidays = os.path.join(data_directory, filename_public_holidays)

In [ ]:
df_public_holidays = pd.read_csv(filepath_public_holidays, delimiter=',', usecols=['Date', 'Holiday'], dtype={'Holiday': str, 'Date': str})
df_public_holidays.sort_values(by='Date', inplace=True)
df_public_holidays['Date'] = pd.to_datetime(df_public_holidays['Date']).dt.date
all_public_holidays = df_public_holidays['Date'].values

df_public_holidays

# 1. Data preparation
Use this section to prepare all the data needed before we can start implementing the model, e.g.
- creating of all possible periods
- cost calculation
- utility calculation

In [ ]:
# The configuration class can be used to store all the configuration parameters
class LocalConfig:
    PLANNING_PERIOD_START_DATE: date = date(2025, 1, 1)
    NUMBER_DAYS_IN_YEAR: int = 365
    MAX_HOLIDAY_PERIOD_LENGTH: int = 30
    
    # Parameters to capture how we derive value from the days off we plan
    ## Control the gain in utility from taking consecutive days off
    MIN_TIME_OFF_TO_GET_VALUE: int = 3
    PERIOD_LENGTH_GAIN_START: int = 4
    PERIOD_LENGTH_GAIN_CUTOFF: int = 20
    
    BASELINE_MARGINAL_VALUE: float = 1
    BONUS_MARGINAL_VALUE: float = 0.5
    
    PREFERRED_WEEKDAYS_OFF: list[int] = [1, 5]
    PREFERRED_DATES_OFF: list[date] = [date(2025, 6, 19)]
    
    HOLIDAY_BUDGET: int = 30

In [ ]:
# Generate all dates in the complete planning period
start_date = LocalConfig.PLANNING_PERIOD_START_DATE
end_date = start_date + timedelta(days=LocalConfig.NUMBER_DAYS_IN_YEAR-1)

all_dates_in_period = [start_date + timedelta(days=i) for i in range(LocalConfig.NUMBER_DAYS_IN_YEAR)]
print(f'Planning holidays between {start_date} and {end_date}')

In [ ]:
def is_weekend_day(day: date) -> bool:
    return day.isoweekday() in [6, 7]

def get_all_weekend_days(all_dates: list[date]) -> list[date]:
    return [day for day in all_dates if is_weekend_day(day)]

# print(is_weekend_day(date(2024, 9, 21)))
# print(get_all_weekend_days(all_dates_in_period))

In [ ]:
# Visualize the distribution of weekend days and holidays in the year
all_weekend_days = get_all_weekend_days(all_dates_in_period)

holiday_calendar_plot(all_public_holidays, all_weekend_days, [])
_ = plt.title(f"Overview of all public holidays and weekends in {year}")

## 1a. Generate all possible holiday periods

In [ ]:
# Pre-generate all possible holiday periods 
def generate_all_possible_holiday_periods(potential_start_dates: list[date], planning_end_date: date) -> list[HolidayPeriod]:
    all_periods = []
    for start_date in potential_start_dates:
        for duration in range(1, LocalConfig.MAX_HOLIDAY_PERIOD_LENGTH + 1):
            if start_date + timedelta(days=duration) > planning_end_date:
                break
            
            new_period = HolidayPeriod(start_date, duration)
            all_periods.append(new_period)
            
    return all_periods


holiday_periods = generate_all_possible_holiday_periods(all_dates_in_period, end_date)

## 1b. Calculate utility we derive from taking a specific holiday period

In [ ]:
# We derive a marginal value from being off on a specific day, which represents the incremental value of being off on that day. For example, being off on Friday brings me some bonus score, as well as being off on specific dates.
def generate_marginal_value_lookup(
    all_dates_in_period: list[date],
) -> dict[date, float]:
    dict_marginal_value = {}
    
    for datestamp in all_dates_in_period:
        dict_marginal_value[datestamp] = LocalConfig.BASELINE_MARGINAL_VALUE
        
        if datestamp.isoweekday() in LocalConfig.PREFERRED_WEEKDAYS_OFF:
            dict_marginal_value[datestamp] += LocalConfig.BONUS_MARGINAL_VALUE
            continue
        
        if datestamp in LocalConfig.PREFERRED_DATES_OFF:
            dict_marginal_value[datestamp] += LocalConfig.BONUS_MARGINAL_VALUE
            continue
            
    return dict_marginal_value
    
dict_marginal_value_of_day_off = generate_marginal_value_lookup(all_dates_in_period)
# dict_marginal_value_of_day_off
    

In [ ]:
# Besides the marginal value of being some time off, we also get additional value for being off for a longer period of time. This is represented by the following step function
def calculate_value_for_period_duration(duration_in_days: int) -> float:
    value = 1
    
    min_duration = LocalConfig.PERIOD_LENGTH_GAIN_START
    max_duration = LocalConfig.PERIOD_LENGTH_GAIN_CUTOFF
    if duration_in_days < min_duration:
        return 0
    
    # Above the minimum holiday period length, we accumulate an exponentially increasing value for the time off.
    if duration_in_days > max_duration:
        additional_value = (max_duration - min_duration + 1) * value
        return additional_value

    additional_value = (duration_in_days - min_duration + 1) * value
    return additional_value

_ = plt.plot([calculate_value_for_period_duration(i) for i in range(1, 30)])
_ = plt.title("Value of taking consecutive days off")

In [ ]:
# Using this function, we can now calculate the value of a specific holiday period as the combination of the marginal value and the period-based value
def calculate_value_of_period(period: HolidayPeriod, marginal_value_lookup: dict[date, float], period_based_value_generator: Callable) -> float:
    total_value = 0
    if period.duration() < LocalConfig.MIN_TIME_OFF_TO_GET_VALUE:
        return 0
    
    for day in period.all_days():
        total_value += marginal_value_lookup[day]
        
    total_value += period_based_value_generator(period.duration())
    return total_value


## 1c. Calculate the cost of taking a specific holiday period

In [ ]:
# We should determine how many units to spend for a specific holiday period. Bank holidays and weekend days are "free" days off, so we do not need to spend any units for them.
def generate_cost_lookup(all_dates_in_period: list[date], cost_free_dates: list[date]) -> dict[date, int]:
    dict_cost = {}
    
    for datestamp in all_dates_in_period:
        if is_weekend_day(datestamp):
            dict_cost[datestamp] = 0
            continue
        
        if datestamp in cost_free_dates:
            dict_cost[datestamp] = 0
            continue
            
        dict_cost[datestamp] = 1
        
    return dict_cost

In [ ]:
def calculate_cost_of_period(period: HolidayPeriod, daily_cost_lookup: dict[date, int]) -> int:
    total_cost = 0
    for day in period.all_days():
        total_cost += daily_cost_lookup[day]
    
    return total_cost

In [ ]:
dict_cost_of_taking_day_off = generate_cost_lookup(all_dates_in_period, all_public_holidays)


# 2. Model implementation
Using these building blocks, we can now implement the optimization model itself.
Check the code snippets in Figma for some assistance.

In [ ]:
model = pulp.LpProblem("HolidayPlanning", pulp.LpMaximize)

## 2a. Decision variable creation

In [ ]:
# Now we are ready to start creating the variables to decide whether we take a specific day off or not
dict_variables = {}
for period in holiday_periods:
    var_name = f"being_off_{period}"
    dict_variables[period] = pulp.LpVariable(cat=pulp.LpBinary, name=var_name)


## 2b. Objective function definition

In [ ]:
# Building the objective function which will aim to maximize the value of the time off
objective_function_elements = []
for period, decision_variable in dict_variables.items():
    value_of_period = calculate_value_of_period(period, dict_marginal_value_of_day_off, calculate_value_for_period_duration)
    new_element = value_of_period * decision_variable
    objective_function_elements.append(new_element)
    
objective_function = pulp.lpSum(objective_function_elements)
model.setObjective(objective_function)


## 2c. Constraints definition

In [ ]:
# We need to make sure that we do not exceed the budget of days off
# So that did not really work ... 
# Let's add a constraint that we can only take a certain number of days off
max_days_off = LocalConfig.HOLIDAY_BUDGET
lhs_elements = []

for period, decision_variable in dict_variables.items():
    cost = calculate_cost_of_period(period, dict_cost_of_taking_day_off)
    lhs_elements.append(decision_variable * cost)
    
budget_constraint = pulp.LpConstraint(e=pulp.lpSum(lhs_elements), sense=pulp.LpConstraintLE, rhs=max_days_off, name='BudgetConstraint')
model.addConstraint(budget_constraint)


In [ ]:
# On its own, this would lead to overlapping periods which is not realistic as we cannot get twice the gain of a single day off. Hence, we will add a set of constraints for each day of the week, indicating it can be covered only once.
dict_date_variable_coverage: dict[date, set[pulp.LpVariable]] = {datestamp: set() for datestamp in all_dates_in_period}

for period, decision_variable in dict_variables.items():
    for datestamp in period.all_days():
        dict_date_variable_coverage[datestamp].add(decision_variable)

dict_date_constraints: dict[date, pulp.LpConstraint] = {}
for datestamp, variables in dict_date_variable_coverage.items():
    type_c = pulp.LpConstraintLE
    
    constraint = pulp.LpConstraint(
        e=pulp.lpSum(variables),
        sense=pulp.LpConstraintLE,
        rhs=1,
        name=f'OneDayPerDate_{datestamp}'
    )
    dict_date_constraints[datestamp] = constraint
    model.addConstraint(constraint)
    

# 3. Model solving and result interpretation

In [ ]:
model.solve(pulp.PULP_CBC_CMD(timeLimit=20))

In [ ]:
if model.status != pulp.LpStatusOptimal:
    print("Model did not find an optimal solution")

In [ ]:
# Extract the solution by checking the holiday periods we should take off based on the model's optimal result
def get_selected_holiday_periods(dict_variables: dict[HolidayPeriod, pulp.LpVariable]) -> list[HolidayPeriod]:
    selected_periods = []
    for period, decision_variable in dict_variables.items():
        if decision_variable.value() is None:
            continue

        if decision_variable.varValue > 0.99:
            selected_periods.append(period)

    return selected_periods

In [ ]:
selected_periods = get_selected_holiday_periods(dict_variables)

print(f"Selected {len(selected_periods)} holiday periods\n")
for period in selected_periods:
    cost = calculate_cost_of_period(period, dict_cost_of_taking_day_off)
    value = calculate_value_of_period(period, dict_marginal_value_of_day_off, calculate_value_for_period_duration)
    print(f"\t{period} -> cost: {cost} \t value: {value}")

In [ ]:
def get_all_days_off(all_periods: list[HolidayPeriod]) -> set[date]:
    all_dates_off = set()
    
    for period in all_periods:
        all_dates_off = all_dates_off.union(set(period.all_days()))
    
    return all_dates_off


In [ ]:
all_dates_off = get_all_days_off(selected_periods)

In [ ]:
holiday_calendar_plot(all_public_holidays, all_weekend_days, all_dates_off)